# TDI Capstone - Final Report

### Abstract:
Phase III clincial trials have a massive effect on the market captalizatizations of pharmaceutical companies. A successful phase III trial for a novel drug allows a company to begin marketing drugs to a previously unserved clinical indication, yielding large revenue streams. In order for a drug to pass it must pass through roughly 10 years of Research and be tested on thousands (or tens of thousands) of patients. 

The long time scales of trials and the sheer numbers of people involved (patients, research scientists, clinicans, research coordinators, goverment regulators...), means that priveleged information regarding trial status has an unusally high level of exposure, compared to other high tech industries. [Recent research](https://academic.oup.com/jnci/article/103/20/1507/904625/Company-Stock-Prices-Before-and-After-Public) suggests that this information exposure can cause detectable movement in this public stock markets. 

If leaks of priveleged information can affect the valuations of pharmaceutical companies, can the movements of these valutaions be identified using Machine Learning and used to inform smarter trading decisions? 

### Gathering Data:

To start with this problem, we need to select a range of target companies (in the pharma sector), find daily closing prices of thier stocks, and get the dates of thier relevant approval announcements. With this information, I'll attempt to extract features and train a machine learning model. 

Data Sources: 
* [fdaTracker.com's free PDUFA Calendar](https://www.fdatracker.com/fda-calendar/)
* [Biopharm Catalyst's upcoming PDUFA Calendar](https://www.biopharmcatalyst.com/calendars/fda-calendar)
* [AlphaVantage's Stock Price API](https://www.alphavantage.co/)

###### First:
First, lets get the historical PDUFA (FDA announcement) Dates:

In [1]:
from urllib2 import urlopen
import ics
import re
from datetime import datetime, timedelta
from alpha_vantage.timeseries import TimeSeries
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import dill
from random import randint

In [2]:
tickerRe = re.compile(r"\A[A-Z]{3,4}\W")
today = datetime.today()

FdaUrl = "https://calendar.google.com/calendar/ical/5dso8589486irtj53sdkr4h6ek%40group.calendar.google.com/public/basic.ics"
FdaCal = ics.Calendar(urlopen(FdaUrl).read().decode('iso-8859-1'))
FdaCal

<Calendar with 552 events>

In [3]:
past_pdufa_syms = set()
for event in FdaCal.events:
    matches = re.findall(tickerRe, event.name)
    if len(matches) >=1:
        eComp = str(matches[0]).strip().strip(".")
        past_pdufa_syms.add(eComp)

In [4]:
print past_pdufa_syms

set(['ENTA', 'AAAP', 'PAR', 'KITE', 'BSTC', 'DNDN', 'ELTP', 'ZSPH', 'GSK', 'CLVS', 'ADLR', 'XENT', 'IRWD', 'VNDA', 'RPTP', 'ACUR', 'DEPO', 'ARIA', 'CHMA', 'VRTX', 'OREX', 'THRX', 'HPTX', 'PCRX', 'ENDP', 'NAVB', 'DRRX', 'OTIC', 'EXEL', 'FLXN', 'ZGEN', 'DSCO', 'SRPT', 'PSDV', 'BCRX', 'MRK', 'ADMP', 'ADMS', 'ISTA', 'AMRN', 'BMY', 'ITMN', 'CEMP', 'KMPH', 'RLYP', 'ANAC', 'SUPN', 'JNJ', 'AERI', 'SNY', 'VION', 'AMAG', 'HZNP', 'REGN', 'SOMX', 'COLL', 'CRTX', 'LPCN', 'VCEL', 'GTXI', 'CTIC', 'LLY', 'AFFY', 'CYPB', 'HGSI', 'OCUL', 'AEGR', 'HOLX', 'OSI', 'DVAX', 'TEVA', 'SGEN', 'TTNP', 'ACOR', 'RPRX', 'EGRX', 'JAZZ', 'AUXL', 'QCOR', 'FLML', 'NEOS', 'AGRX', 'RARE', 'RDUS', 'NGSX', 'RMTI', 'NPSP', 'CLDA', 'DYAX', 'ALTH', 'ARLZ', 'VVUS', 'LXRX', 'NVO', 'MELA', 'ANX', 'SCMP', 'POZN', 'HRTX', 'ACAD', 'AGIO', 'TSRO', 'RGEN', 'FURX', 'SPPI', 'AMGN', 'ONXX', 'BMTI', 'BPAX', 'REPH', 'DOR', 'NEW', 'WCRX', 'ASTX', 'KTOV', 'ISIS', 'AVEO', 'BIOD', 'GNBT', 'VRX', 'IGXT', 'LGND', 'MDVN', 'AMLN', 'OMER', 'KMDA', 

In [5]:
av_key_handle = open("alphavantage.apikey", "r")
ts = TimeSeries(key=av_key_handle.read().strip(), output_format='pandas')
av_key_handle.close()

In [6]:
dataframes = dict()
value_errors = set()
other_errors = set()
for ticker in tqdm_notebook(past_pdufa_syms):
    try:
        df, meta = ts.get_daily(symbol=ticker, outputsize='full')
        dataframes[meta["2. Symbol"]] = df
    except ValueError:
        value_errors.add(ticker)
    except:
        other_errors.add(ticker)

In [7]:
print value_errors
print other_errors

set(['VION', 'FURX', 'DNDN', 'GEVA', 'BPAX', 'INSV', 'ZSPH', 'WCRX', 'ADLR', 'CBRX', 'PPDI', 'ISIS', 'CYPB', 'HGSI', 'PCYC', 'THRX', 'MDVN', 'HPTX', 'APPA', 'SLXP', 'SNTS', 'AVNR', 'AUXL', 'TSPT', 'QCOR', 'FLML', 'NEOL', 'ZGEN', 'DSCO', 'XNPT', 'SVNT', 'ALXA', 'FRX', 'NGSX', 'ISTA', 'NPSP', 'CLDA', 'BIOD', 'DYAX', 'CHTP', 'ITMN', 'RLYP', 'ANAC', 'DRTX', 'KYTH', 'MELA', 'ANX', 'POZN'])
set([])


In [8]:
dill.dump(dataframes, open('final_raw_dataframe_dict.pkl', 'w'))

###### Mini Checkpoint for slow API calls

In [9]:
dataframes = dill.load(open('final_raw_dataframe_dict.pkl', 'r'))

Now we'll run through our past FDA dates and join the FDA actions to each dataframe

In [10]:
company_list = dataframes.keys()

In [11]:
price_and_fda = dict()
for company in tqdm_notebook(company_list):
    company_events = []
    for event in FdaCal.events:
        matches = re.findall(tickerRe, event.name)
        if len(matches)>=1:
            if company in matches[0]:
                company_events.append((event.begin.datetime.strftime("%Y-%m-%d"), True))
    price = dataframes[company]
    raw_dates = pd.DataFrame(company_events, columns = ["date", "pdufa?"])
    dates = raw_dates.set_index("date")
    final = price.join(dates,rsuffix='_y')
    final['pdufa?'].fillna(value=False, inplace = True)
    price_and_fda[company] = final

That leaves us with a dict of dataframes containing every company's stock price, and FDA action dates

In [12]:
price_and_fda['ENTA'].head(3)

,volume,close,high,open,low,pdufa?
2013-03-21,1763600.0,17.18,17.85,14.51,14.31,False
2013-03-22,75200.0,16.81,17.57,17.57,16.71,False
2013-03-25,24100.0,16.83,17.40,17.40,16.80,False


Now that I've got a good crop of downloaded data, lets cache it for good measure. 

In [13]:
dill.dump(price_and_fda, open("final_Prices_and_PDUFAs.pkl", "w"))

### Checkpoint 1 - FDA Action Dates Joined to Equity Prices

In [14]:
price_and_fda = dill.load(open("final_Prices_and_PDUFAs.pkl", "r"))

So thats every company's stock prices, with PDUFA dates going back to around 2006, and pricing data going back to 2001. More than enough data for our analysis. 

Lets unify all the prices into one frame, to construct a pharmaceutical price index. This will give us a base price to normalize our stock prices against, insulating our model from general economic events (the '08 housing crash) or events affecting the whole pharmaceutical sector (passage of new FDA regulations). 

In [15]:
first = True
for ticker, comp_df in price_and_fda.iteritems():
    if first:
        market_df = comp_df.copy()
        market_df.columns = ["volume-"+ticker,
                             "close-"+ticker,
                             "high-"+ticker,
                             "open-"+ticker,
                             "low-"+ticker,
                             "pdufa?-"+ticker]
        first = False
    else:
        market_df = pd.merge(market_df, comp_df, how='outer', left_index=True, right_index=True, suffixes=('', '-'+ticker))

In [16]:
price_mean = market_df.filter(regex='close').mean(axis = 1, skipna = True)
price_stdv = market_df.filter(regex='close').std(axis = 1, skipna = True)

In [17]:
stats_df = pd.merge(price_mean.to_frame(),
                    price_stdv.to_frame(), 
                    left_index=True, 
                    right_index=True, 
                    how='inner')
stats_df.rename(columns={u'0_x':"CP_mean", u'0_y':"CP_stdv"}, inplace=True)

In [18]:
stats_df.head()

,CP_mean,CP_stdv
2000-01-03,26.047255,28.436582
2000-01-04,24.696149,27.097988
2000-01-05,24.816816,27.233825
2000-01-06,25.208395,27.425298
2000-01-07,27.311931,30.537949


This is as good a place as any to cache the closing price index

In [19]:
dill.dump(stats_df, open("close_price_stats_frame_final.pkl", "w"))

### Checkpoint 2

In [20]:
stats_df = dill.load(open("close_price_stats_frame_final.pkl", "r"))

Now I have the mean and standard deviation of close prices (`stats_df`) for every day of my data coverage. This will make it easy to normalize prices for every slice of time relevant to an FDA trial. 

Time to cut time slices for each clinical trial and generate a population of clinical trials and normalized prices.

In [21]:
norm_data = []
for company in tqdm_notebook(company_list):
    df = price_and_fda[company].join(stats_df, how='left').reset_index()
    pdufa_dates = df.index[df['pdufa?']].tolist()
    if len(pdufa_dates) > 0:
        for date in pdufa_dates:
            pRange = range(date-120, date-7)
            pCloses, pVolumes = [], []
            for i in pRange:
                try:
                    close_price = df.loc[i]['close']
                    volume = df.loc[i]['volume']
                    mean_price = df.loc[i]['CP_mean']
                    stdv_price = df.loc[i]['CP_stdv']
                    pCloses.append(( df.loc[i]['index'],(close_price-mean_price)/(stdv_price) ))
                    pVolumes.append(( df.loc[i]['index'], volume ))
                except:
                    pCloses.append(None)
                    pVolumes.append(None)
            norm_data.append((company, df.loc[date]['index'], (pCloses, pVolumes)))

Well we have normalized slices, lets add the annotations from our score sheet

In [22]:
scores = [line.split() for line in open("score_sheet_complete.txt", "r").readlines()]

In [23]:
norm_data_annotated = []
mismatches = []
for datum in tqdm_notebook(norm_data):
    for score in scores:
        if datum[0] == score [0] and datum [1] == score[1]:
            norm_data_annotated.append((datum[0], datum[1], score[2], datum[2] ))
            break

In [24]:
dill.dump(norm_data_annotated, open("normalized_training_data.pkl", "w"))

### Checkpoint 3

In [25]:
norm_data_annotated = dill.load(open("normalized_training_data.pkl", "r"))

Now we have normalized stock prices, in 120-7 day slices prior to FDA action dates. Lets pull those back into smaller pandas frames for feature extraction. 

In [26]:
def assemble_frame(datum):
    df = pd.DataFrame(datum[3][0], columns=['date','norm_price'])
    df['event'] = datum[0]+"/"+datum[1]
    df['outcome'] = int(datum[2])
    return df

In [27]:
first = True

for line in tqdm_notebook(norm_data_annotated):
    try:
        if first:
            agg_data = assemble_frame(line)
            first = False
        else:
            tmp_data = assemble_frame(line)
            agg_data = pd.concat([agg_data, tmp_data],ignore_index=True)
    except:
        print line[0], line[1], "failed"

COLL 2015-10-12 failed
NEOS 2015-11-09 failed



In [28]:
agg_data['date_stamp'] = pd.to_datetime(agg_data['date'])
event_labels = pd.factorize(agg_data['event'])
agg_data["event_stamp"] = event_labels[0]

Now lets remove out the trials will null prices on some days (either due to acquisitions or bankruptcies). 

In [29]:
agg_data['null'] = pd.isnull(agg_data).apply(lambda x: sum(x) , axis=1)
cleaned_agg = agg_data[agg_data['null'] == 0]

In [30]:
cleaned_agg.head()

,date,norm_price,event,outcome,date_stamp,event_stamp,null
0,2015-12-08,-0.125217,AAAP/2016-06-01,1,2015-12-08,0,0
1,2015-12-09,-0.119686,AAAP/2016-06-01,1,2015-12-09,0,0
2,2015-12-10,-0.118359,AAAP/2016-06-01,1,2015-12-10,0,0
3,2015-12-11,-0.110503,AAAP/2016-06-01,1,2015-12-11,0,0
4,2015-12-14,-0.102528,AAAP/2016-06-01,1,2015-12-14,0,0


In [31]:
dill.dump(cleaned_agg, open('final_cleaned_price_slices.pkl', 'w'))

### Checkpoint 3 - Training data preprocessed

In [32]:
cleaned_agg = dill.load(open('final_cleaned_price_slices.pkl', 'r'))

That's a ready to extract package of every clinical trial scraped. Lets go ahead and make up a test and train split now, while its easy and convinent.

In [33]:
from sklearn.cross_validation import train_test_split

/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
train_data, test_data = train_test_split(norm_data_annotated, train_size = .9)

In [35]:
first = True

for line in tqdm_notebook(train_data):
    try:
        if first:
            train_df = assemble_frame(line)
            first = False
        else:
            tmp_df = assemble_frame(line)
            train_df = pd.concat([train_df, tmp_df],ignore_index=True)
    except:
        print line[0], line[1], "failed"

train_df['date_stamp'] = pd.to_datetime(train_df['date'])
event_labels = pd.factorize(train_df['event'])
train_df["event_stamp"] = event_labels[0]

train_df['null'] = pd.isnull(train_df).apply(lambda x: sum(x) , axis=1)
train_clean = train_df[train_df['null'] == 0]

COLL 2015-10-12 failed
NEOS 2015-11-09 failed



In [36]:
first = True

for line in tqdm_notebook(test_data):
    try:
        if first:
            test_df = assemble_frame(line)
            first = False
        else:
            tmp_df = assemble_frame(line)
            test_df = pd.concat([test_df, tmp_df],ignore_index=True)
    except:
        print line[0], line[1], "failed"
test_df['date_stamp'] = pd.to_datetime(test_df['date'])
event_labels = pd.factorize(test_df['event'])
test_df["event_stamp"] = event_labels[0]

test_df['null'] = pd.isnull(test_df).apply(lambda x: sum(x) , axis=1)
test_clean = test_df[test_df['null'] == 0]

Thats two parts of a bifurcated dataframe. May as well cache it. 

In [37]:
dill.dump(train_clean, open("final_train_df.pkl", "w"))
dill.dump(test_clean, open("final_test_df.pkl", "w"))

### Checkpoint 4 - Test Train Split

In [38]:
train_clean = dill.load(open("final_train_df.pkl", "r"))
test_clean = dill.load(open("final_test_df.pkl", "r"))

Now for the serious work, extracting features from the pricing data in each case. 

I'll be using [tsfresh](http://tsfresh.readthedocs.io/en/latest/text/quick_start.html) to do the hard computing here, and then selecting the most relevant features. While I am able to compute almost 800 features for these data points, I'm going to narrow down to around ten of the most meaningful or important features. 

In [39]:
from tsfresh import extract_features

/opt/conda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [40]:
train_feats = extract_features(train_clean[['norm_price', 'event_stamp', 'date_stamp']], 
                              column_id="event_stamp", column_sort="date_stamp", 
                              column_value="norm_price", n_jobs=0).dropna(axis=1)

Feature Extraction: 100%|██████████| 212/212 [00:00<00:00, 30191.24it/s]


In [41]:
train_feats.head()

variable,norm_price__abs_energy,norm_price__absolute_sum_of_changes,"norm_price__agg_autocorrelation__f_agg_""mean""","norm_price__agg_autocorrelation__f_agg_""median""","norm_price__agg_autocorrelation__f_agg_""var""","norm_price__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","norm_price__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","norm_price__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","norm_price__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","norm_price__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,norm_price__time_reversal_asymmetry_statistic__lag_1,norm_price__time_reversal_asymmetry_statistic__lag_2,norm_price__time_reversal_asymmetry_statistic__lag_3,norm_price__value_count__value_-inf,norm_price__value_count__value_0,norm_price__value_count__value_1,norm_price__value_count__value_inf,norm_price__value_count__value_nan,norm_price__variance,norm_price__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
0,158.826529,2.191306,0.194683,0.185735,0.115427,1.202333,0.261442,0.004621,0.005394,1.313276,...,0.002048,0.006176,0.014571,0.0,0.0,0.0,0.0,0.0,0.006205,0.0
1,110.104474,1.710732,0.339156,0.255810,0.089518,1.087026,-0.829158,-0.013284,0.002832,1.112564,...,-0.001434,-0.004229,-0.008301,0.0,0.0,0.0,0.0,0.0,0.002936,0.0
2,25.876205,0.548592,-0.089333,-0.219019,0.392978,-0.470053,0.123129,0.000654,0.001667,-0.430898,...,0.000328,0.000634,0.000871,0.0,0.0,0.0,0.0,0.0,0.000351,0.0
3,7.586910,5.436295,0.032363,0.050624,0.011449,-0.178790,0.161090,0.001697,0.003289,-0.130614,...,-0.000177,-0.000284,0.000253,0.0,0.0,0.0,0.0,0.0,0.005681,0.0
4,14.381392,1.117137,0.126168,-0.023299,0.101489,-0.325935,0.034650,0.000471,0.004292,-0.307977,...,0.000149,0.000139,0.000090,0.0,0.0,0.0,0.0,0.0,0.001598,0.0


In [42]:
train_y =\
train_df[['event_stamp', 'outcome']]\
.groupby('event_stamp')\
.head(1).set_index('event_stamp')['outcome']

In [43]:
train_y.head()

event_stamp
0    0
1    1
2    1
3    0
4    0
Name: outcome, dtype: int64

In [44]:
test_feats = extract_features(test_clean[['norm_price', 'event_stamp', 'date_stamp']], 
                              column_id="event_stamp", column_sort="date_stamp", 
                              column_value="norm_price", n_jobs=0).dropna(axis=1)

Feature Extraction: 100%|██████████| 24/24 [00:00<00:00, 23119.73it/s]


In [45]:
test_feats.shape

(24, 622)

In [46]:
test_y =\
test_df[['event_stamp', 'outcome']]\
.groupby('event_stamp')\
.head(1).set_index('event_stamp')['outcome']

In [47]:
test_y.shape

(24,)

In [48]:
dill.dump(train_feats, open('final_train_features.pkl','w'))
dill.dump(test_feats, open('final_test_features.pkl','w'))

### Checkpoint 4 - Extracted Features

In [49]:
train_feats = dill.load(open("final_train_features.pkl", "r"))
test_feats = dill.load(open("final_test_features.pkl", "r"))

Now its time to pick out 10 or so meaningful features from the 622 possible features. Time for some reading. Then itll be time to apply those to a classification model. 

In [50]:
print"\n".join(list(train_feats.columns.values))

norm_price__abs_energy
norm_price__absolute_sum_of_changes
norm_price__agg_autocorrelation__f_agg_"mean"
norm_price__agg_autocorrelation__f_agg_"median"
norm_price__agg_autocorrelation__f_agg_"var"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"rvalue"
norm_price__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"slope"
norm_price__agg_li

In [51]:
features_of_interest = ['norm_price__mean',
                        'norm_price__median',
                        'norm_price__mean_change',
                        #'norm_price__mean_abs_change',
                        'norm_price__first_location_of_maximum',
                        'norm_price__first_location_of_minimum',
                        'norm_price__linear_trend__attr_"slope"',
                        'norm_price__count_above_mean',
                        'norm_price__count_below_mean'
                       ]

In [52]:
print train_feats[features_of_interest].shape
train_feats[features_of_interest].head()

(212, 8)


variable,norm_price__mean,norm_price__median,norm_price__mean_change,norm_price__first_location_of_maximum,norm_price__first_location_of_minimum,"norm_price__linear_trend__attr_""slope""",norm_price__count_above_mean,norm_price__count_below_mean
id,,,,,,,,
0,1.182937,1.204903,0.000382,0.389381,0.035398,0.001512,74.0,39.0
1,0.985616,0.985067,-0.000494,0.132743,0.734513,-0.001266,56.0,57.0
2,-0.478165,-0.482909,0.000343,0.415929,0.115044,0.000130,45.0,68.0
3,-0.247912,-0.244275,0.000039,0.805310,0.079646,0.000593,61.0,52.0
4,-0.354502,-0.365625,0.000786,0.991150,0.796460,-0.000441,50.0,63.0


In [53]:
print test_feats[features_of_interest].shape
test_feats[features_of_interest].head()

(24, 8)


variable,norm_price__mean,norm_price__median,norm_price__mean_change,norm_price__first_location_of_maximum,norm_price__first_location_of_minimum,"norm_price__linear_trend__attr_""slope""",norm_price__count_above_mean,norm_price__count_below_mean
id,,,,,,,,
0,0.271306,0.036752,-0.011813,0.044248,0.946903,-0.013196,31.0,82.0
1,-0.394087,-0.385822,0.000021,0.610619,0.557522,-0.000048,68.0,45.0
2,0.584367,0.580125,-0.000229,0.274336,0.814159,-0.000379,51.0,62.0
3,0.059269,0.061939,-0.000667,0.681416,0.973451,0.000506,59.0,54.0
4,-0.368157,-0.377973,-0.001891,0.389381,0.964602,-0.002048,53.0,60.0


Thats our split data, with our features of interest. Lets begin Modeling. 

In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

In [55]:
scaler = StandardScaler()
classifier = SVC(C=1, coef0=1, degree=1)
params = {"C":range(1,5),
          "degree":range(1,3),
          "coef0":range(1,3)
         }
classifier_gs = GridSearchCV(classifier, params)

In [56]:
classifier_gs.fit(scaler.fit_transform(train_feats), train_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=1, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 2, 3, 4], 'coef0': [1, 2], 'degree': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [57]:
classifier_gs.best_params_

{'C': 3, 'coef0': 1, 'degree': 1}

In [58]:
cross_val_score(classifier, scaler.transform(test_feats), y=test_y)

array([ 0.625,  0.625,  0.625])

Thats a trained and cross validated model. Lets pickle it for safe keeping.

In [59]:
dill.dump(classifier, open("final_trained_svc.pkl","w"))

### Checkpoint 5 - Trained Model

In [60]:
classifier = dill.load(open("final_trained_svc.pkl","r"))

## That's all Folks?

At this point, the dirty work has been done.

The model exists, at this point I'm going to work on a visualization to explain how it preforms. That will fufill my work for TDI.

Eventually I plan to create a pipeline that can be deployed on a webapp, but this will take some time, and require some skills I'll have to learn along the way. 

Now that we have a working predictor, lets play with some visualizations to show how powerful is can be.

To start, lets collect our data into one large frame to explore (actually, run the same processing steps on the non-split data).

In [61]:
all_feats = extract_features(cleaned_agg[['norm_price', 'event_stamp', 'date_stamp']], 
                              column_id="event_stamp", column_sort="date_stamp", 
                              column_value="norm_price", n_jobs=0).dropna(axis=1)

Feature Extraction: 100%|██████████| 236/236 [00:00<00:00, 31440.96it/s]


In [62]:
cleaned_agg

,date,norm_price,event,outcome,date_stamp,event_stamp,null
0,2015-12-08,-0.125217,AAAP/2016-06-01,1,2015-12-08,0,0
1,2015-12-09,-0.119686,AAAP/2016-06-01,1,2015-12-09,0,0
2,2015-12-10,-0.118359,AAAP/2016-06-01,1,2015-12-10,0,0
3,2015-12-11,-0.110503,AAAP/2016-06-01,1,2015-12-11,0,0
4,2015-12-14,-0.102528,AAAP/2016-06-01,1,2015-12-14,0,0
5,2015-12-15,-0.116008,AAAP/2016-06-01,1,2015-12-15,0,0
6,2015-12-16,-0.135902,AAAP/2016-06-01,1,2015-12-16,0,0
7,2015-12-17,-0.127469,AAAP/2016-06-01,1,2015-12-17,0,0
8,2015-12-18,-0.084903,AAAP/2016-06-01,1,2015-12-18,0,0
9,2015-12-21,-0.124031,AAAP/2016-06-01,1,2015-12-21,0,0


In [63]:
print all_feats[features_of_interest].shape
all_feats[features_of_interest].head()

(236, 8)


variable,norm_price__mean,norm_price__median,norm_price__mean_change,norm_price__first_location_of_maximum,norm_price__first_location_of_minimum,"norm_price__linear_trend__attr_""slope""",norm_price__count_above_mean,norm_price__count_below_mean
id,,,,,,,,
0,0.009741,0.048816,0.000875,0.610619,0.203540,0.002459,60.0,53.0
1,0.059269,0.061939,-0.000667,0.681416,0.973451,0.000506,59.0,54.0
2,0.096399,0.077117,0.001571,0.893805,0.681416,0.000712,35.0,78.0
3,0.394134,0.375288,0.002183,0.699115,0.336283,0.004218,53.0,60.0
4,0.665529,0.664498,0.001029,0.920354,0.530973,0.001494,54.0,59.0


In [64]:
all_y =\
cleaned_agg[['event_stamp', 'outcome']]\
.groupby('event_stamp')\
.head(1).set_index('event_stamp')['outcome']

In [65]:
all_events =\
cleaned_agg[['event_stamp','event']]\
.groupby('event_stamp')\
.head(1).set_index('event_stamp')['event']

In [66]:
all_predictions = classifier_gs.predict(scaler.transform(all_feats))

In [67]:
events_and_predictions = pd.DataFrame(all_events).join(pd.DataFrame(all_predictions))

In [68]:
events_and_predictions.shape

(236, 2)

Lets generate a random population of guesses to check, In this case, I'm going to generate random 1,000,000 guesses (of 1 or 0) for each row of the data and average these to generate a random distribution. 

In [69]:
random_guesses = np.random.randint(0,2,size=(events_and_predictions.shape[0], 1000000))

In [70]:
print random_guesses.shape
random_guesses

(236, 1000000)


array([[0, 0, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 0],
       ..., 
       [0, 0, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 1, 0]])

In [71]:
random_guess_means = np.mean(random_guesses, axis = 1)

In [72]:
random_guess_means.shape

(236,)

In [73]:
events_and_predictions.columns = ['event', 'pass_prediction']
events_and_predictions['pass_random'] = [int(x) for x in random_guess_means.round()]
events_and_predictions['pass_all'] = 1

In [74]:
events_and_predictions.head(3)

,event,pass_prediction,pass_random,pass_all
event_stamp,,,,
0,AAAP/2016-06-01,0,0,1
1,AAAP/2016-12-28,0,1,1
2,AAAP/2017-08-28,0,0,1


In [75]:
predicted_passes = events_and_predictions[events_and_predictions['pass_prediction'] == 1]
predicted_fails = events_and_predictions[events_and_predictions['pass_prediction'] == 0]

Thats every event in a human readable format, with it's machine predicted outcome, a randomly guessed outcome, and an all outcomes guess.
Now lets pull some price data and see how each model fares.

First, lets run over our inital dataframe of prices, and grab the prices of interest, the prices from one week before the PDFUA Date and about two months after the PDUFA date. I included the (ugly) try except block to tolerate weekends and market closure days.

In [76]:
def x_days_later(date_str, x):
    pdufa_day = datetime.strptime(date_str,"%Y-%m-%d")
    change = timedelta(days = x)
    delta_date = pdufa_day + change
    return delta_date.strftime("%Y-%m-%d")

In [77]:
lead_price = 7 #how many days before the PDUFA to sample the price history
lag_price = 60 #how many days after the PDUFA to sample the price history
prior_and_post_prices = []
mindate = datetime(9999, 12, 31)
maxdate = datetime(1, 1, 1)
for stamp in events_and_predictions['event']:
    ticker, date = stamp.split("/")
    if datetime.strptime(date,"%Y-%m-%d") < mindate:
        mindate = datetime.strptime(date,"%Y-%m-%d")
    if datetime.strptime(date,"%Y-%m-%d") > maxdate:
        maxdate = datetime.strptime(date,"%Y-%m-%d")
    try:
        p_7_day = price_and_fda[ticker].loc[x_days_later(date, -1*lead_price)]['close']
    except KeyError:
        p_7_day = None
    try:
        p_60_day = price_and_fda[ticker].loc[x_days_later(date,lag_price)]['close']
    except KeyError:
        try:
            p_60_day = price_and_fda[ticker].loc[x_days_later(date,lag_price-1)]['close']
        except KeyError:
            try:
                p_60_day = price_and_fda[ticker].loc[x_days_later(date,lag_price-2)]['close']
            except KeyError:
                try:
                    p_60_day = price_and_fda[ticker].loc[x_days_later(date,lag_price-3)]['close']
                except KeyError:
                    p_60_day = None
    line = (stamp, p_7_day, p_60_day)
    if None not in line:
        prior_and_post_prices.append(line)
print mindate
print maxdate

2006-10-06 00:00:00
2017-10-06 00:00:00


In [78]:
prior_and_post_prices = pd.DataFrame(prior_and_post_prices)
prior_and_post_prices.columns = ['event', 'close_-7_Day', 'close_+'+str(lag_price)+'_Day']
prior_and_post_prices.head(3)

,event,close_-7_Day,close_+60_Day
0,AAAP/2016-06-01,29.29,31.29
1,AAAP/2016-12-28,24.68,36.96
2,AAAP/2017-08-28,48.65,72.91


In [79]:
predictions_and_prices =\
pd.merge(events_and_predictions, prior_and_post_prices, on='event')

In [80]:
def get_date_from_stamp(stamp):
    return datetime.strptime(stamp.split("/")[1],"%Y-%m-%d")

In [81]:
predictions_and_prices['date'] = predictions_and_prices.event.apply(get_date_from_stamp)

In [82]:
predictions_and_prices['price_%_change'] = \
((predictions_and_prices['close_+60_Day']-predictions_and_prices['close_-7_Day']) / \
 predictions_and_prices['close_-7_Day'])*100

In [83]:
sim_df = predictions_and_prices.sort_values(['date'], axis=0).dropna(axis=0).set_index('date')

In [84]:
sim_df.head(10)

,event,pass_prediction,pass_random,pass_all,close_-7_Day,close_+60_Day,price_%_change
date,,,,,,,
2006-10-06,NEW/2006-10-06,0,0,1,12.30,12.0000,-2.439024
2007-08-15,SPPI/2007-08-15,0,0,1,4.00,4.5400,13.500000
2007-08-21,NBIX/2007-08-21,0,0,1,10.31,10.0200,-2.812803
2007-12-12,NBIX/2007-12-12,0,0,1,11.72,5.1000,-56.484642
2008-01-30,PGNX/2008-01-30,0,1,1,18.04,6.2100,-65.576497
2008-02-29,JAZZ/2008-02-29,0,0,1,11.52,9.0000,-21.875000
2008-04-30,PGNX/2008-04-30,1,0,1,8.03,16.3800,103.985056
2008-06-19,LGND/2008-06-19,0,1,1,2.80,3.3800,20.714286
2008-12-01,SOMX/2008-12-01,0,0,1,9.12,19.8392,117.535088


That looks like enough info for a basic viz describing the financial preformance of this algorithm

In [85]:
dill.dump(sim_df, open("final_sim_df.pkl", "w"))

### Checkpoint 6 dataframe for portfolio simulation

In [86]:
sim_df = dill.load(open("final_sim_df.pkl", "r"))

In [87]:
mln_changes = []
rnd_changes = []
all_changes = []
for date in sim_df.iterrows():
    info = date[1]
    if info['pass_prediction'] == 1:
        mln_changes.append(info['price_%_change'])
    else:
        mln_changes.append(0.0)
    if info['pass_random'] == 1:
        rnd_changes.append(info['price_%_change'])
    else:
        rnd_changes.append(0.0)
    if info['pass_all'] == 1:
        all_changes.append(info['price_%_change'])
    else:
        all_changes.append(0.0)

Now we have a list of every percentage change in value we would have if we were to invest over the 10 year analysis period using each methodology: (<font color='SteelBlue'>support vector machine selection</font>, <font color='SeaGreen'>randomly buying stocks</font>, or <font color='Tomato'>buy all stocks with upcoming clinical trials</font>). This projection approximates the change in portfolio value if every equity is purchased one week prior to a PDUFA date and sold two months following the PDUFA date. 

In the cell below, you can set the inital investment, and a graph will be generated showing the percentage increase over ten years of trading on pharmaceutical stocks. Line thicknesses are assigned based on `log(starting_dollars)`.

In [88]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from math import log
output_notebook()
def calc_changes(start, events):
    prices = [start]
    for event in events:
        last_price = prices[-1]
        prices.append(last_price+(last_price*(event/100)))
    return prices

Loading BokehJS ...

In [89]:
starting_dollars = [50, 100] #a list of staring values for each investment strategy
p = figure(plot_width=500, 
           plot_height=500, 
           x_axis_label="# of trials traded on",
           y_axis_label="approximate portfolio value"
          )
for start in starting_dollars:
    line_y = calc_changes(start, mln_changes)
    line_x = [x for x in range(len(line_y))]
    p.line(line_x,
           line_y,
           line_width=log(start, 10),
           color = "SteelBlue"
          )
    
    line_y = calc_changes(start, rnd_changes)
    line_x = [x for x in range(len(line_y))]
    p.line(line_x,
           line_y,
           line_width=log(start, 10),
           color = "Tomato"
          )
    
    line_y = calc_changes(start, all_changes)
    line_x = [x for x in range(len(line_y))]
    p.line(line_x,
           line_y,
           line_width=log(start, 10),
           color = "seagreen"
          )
show(p)

This plot shows the percentage change in portfolio value over time, This plot assumes that all profits are reinvested into upcoming PDUFA equities based on thier strategy (<font color='SteelBlue'>support vector machine selection</font>, <font color='SeaGreen'>randomly buying stocks</font>, or <font color='Tomato'>buy all stocks with upcoming clinical trials</font>).  

# Closing Remarks

The ability of a Machine learning model to predict the failure or passage of drugs through the FDA, based on statistically extracted features, suggests that fundamental information regarding pharmaceutical approvals may be more exposed than in other industries. One obvious application of these techniques is informing trading decisions in the pharmaceutical space (or another industry with similarly high information exposure). This model could also be applied to other industries as a measure of information exposure and its effects on the public markets. 

I think the obvious next step for this project is to package the model into a webapp, that makes buying and short-selling reccomendations based on upcoming FDA action dates. Unfortunately, my web development skillset will need some buffing up to fully realize this ambition. 